In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from scipy.io import mmread
sys.path.append(os.path.abspath(os.path.join('..', 'utils')))
import geo_utils
from geo_utils import read_prefixes_from_metadata_txt

root_path = '/var/datasets/single_papers/2023_feargal/GSE199750'


In [ ]:
metadata_df = read_prefixes_from_metadata_txt(f'{root_path}/GSE199750_series_matrix.txt')
print(metadata_df.shape)
metadata_df.head()